**Rationale** Hypothesis testing is foundational to the entire field of statistics. Without the Central Limit Theorem and hypothesis testing, we would not have modern day (social) science. In marketing, this is generally most important when doing A/B testing of ads. In this assignment, you will practice computing basic summary statistics and conducting hypothesis tests. 

[Datasets](https://drive.google.com/drive/folders/1D6D-zRU3oiP12c0dFMQjs66JDhBZBtax?usp=sharing) required:
1. [FB ad campaign data](https://drive.google.com/file/d/1JfwvumxS2oys8ZcQtsb8GGmeRfnkzFdy/view?usp=sharing).
1. [Starbucks Promos](https://drive.google.com/file/d/1W7x5_PU5KszT8mqXqqEmRfXuR8CRr1DX/view?usp=sharing)

# 1. (2 points) FB Ad campaigns

1. Use a groupby operation create a dataframe called `sumstats` consisting of the total impressions and clicks for each `xyz_campaign_id`.
1. Using `sumstats`, create a new column called `ctr` that represents the [click through rate](https://support.google.com/google-ads/answer/2615875?hl=en) for each campaign. Which campaign had the highest CTR?
1. Create a column for `sd` for standard deviation of the click through rate. 

Hint: Recall from the notes that the standard deviation of a binomial distribution with $Pr(Success) = p$ is $SD(p) = \sqrt{p(1-p)}$. So if the probability of a binary outcome (such as clicking on an ad) is observed to be .3, the standard deviation is $\sqrt{.3(.7)}=\sqrt{.21}$.

In [1]:
# imports and mount google drive
import os, pandas as pd, numpy as np
from scipy import stats

from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
# set the path to the datasets for A5
fpath = 'drive/MyDrive/MKTG3509/Data Sets'
os.listdir(fpath)

['quebec_biz.csv',
 'quebec_yelp_reviews.csv',
 'goodDataSet.csv',
 'renttherunway_final_data.json',
 'telecom_churn.csv',
 'starbucks_promos.csv',
 'facebook_ads.csv']

In [3]:
# read in the facebook_ads.csv file as the dataframe variable named fb
fb = pd.read_csv('drive/MyDrive/MKTG3509/Data Sets/facebook_ads.csv')

In [4]:
# Take a look at the first 5 rows
# Observe that each row represents 1 ad from 1 campaign and a target demographic group (age, gender, interest)
fb.head()

,ad_id,xyz_campaign_id,fb_campaign_id,age,gender,interest,Impressions,Clicks,Spent,Total_Conversion,Approved_Conversion
0,708746,916,103916,30-34,M,15,7350,1,1.43,2,1
1,708749,916,103917,30-34,M,16,17861,2,1.82,2,0
2,708771,916,103920,30-34,M,20,693,0,0.00,1,0
3,708815,916,103928,30-34,M,28,4259,1,1.25,1,0
4,708818,916,103928,30-34,M,28,4133,1,1.29,1,1


In [5]:
# use a groupby to create new dataframe, sumstats that tabulates the total clicks and impressions for each campaign.
# Generic groupby syntax: sumstats = df.groupby('groupbyvariable')[['summary_variable1', 'summary_variable2']].summaryfunction().reset_index()
# use the correct dataframe name, variables (i.e. column names) and summaryfunction
sumstats = fb.groupby('xyz_campaign_id')['Clicks','Impressions'].sum().reset_index()
sumstats.head()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


,xyz_campaign_id,Clicks,Impressions
0,916,113,482925
1,936,1984,8128187
2,1178,36068,204823716


In [6]:
# create a new column in sumstats called 'ctr' (short for click through rate)
# recall that ctr = clicks / impressions
sumstats['ctr'] = (sumstats['Clicks']/sumstats['Impressions'])
sumstats.head()



,xyz_campaign_id,Clicks,Impressions,ctr
0,916,113,482925,0.000234
1,936,1984,8128187,0.000244
2,1178,36068,204823716,0.000176


In [7]:
# take a look at the whole sumstats dataframe
sumstats


,xyz_campaign_id,Clicks,Impressions,ctr
0,916,113,482925,0.000234
1,936,1984,8128187,0.000244
2,1178,36068,204823716,0.000176


In [8]:
# create a new column ctr_sd for the std. dev of the ctr (See hint above)
# pay attention to PEMDAS, esp. that you want to take the sqrt after computing p*(1-p)
# np.sqrt(x) returns the sqrt of some number x
# alternatively, x**.5 also returns the sqrt of x
sumstats['ctr_sd'] = np.sqrt(sumstats.ctr * (1-sumstats.ctr))
sumstats.head()


,xyz_campaign_id,Clicks,Impressions,ctr,ctr_sd
0,916,113,482925,0.000234,0.015295
1,936,1984,8128187,0.000244,0.015621
2,1178,36068,204823716,0.000176,0.013269


In [ ]:
# take another look at the whole sumstats dataframe
# compare the ctr_std to calculating it by hand, make sure you did this correctly.



In [9]:
sumstats[sumstats['ctr']==sumstats['ctr'].max()]

,xyz_campaign_id,Clicks,Impressions,ctr,ctr_sd
1,936,1984,8128187,0.000244,0.015621


## Answer (edit this cell)
The campaign with the highest CTR is: **936**


# 2. (4 points) Hypothesis testing and confidence intervals

1. Compute the 95% confidence interval for the CTR for each campaign. Compare the confidence intervals of campaign 916 and 936. What can you conclude about the relative perfrmance of the 2 ads in the population (e.g. are they very different, similar, etc.)? How about 916 vs. 1178?
1. Was campaign 936 statistically different compared to campaign 916? How about 936 vs. 1178? Use the `ttest_2sample` function from the notes. Remember, you must define it in your Colab session in order to use it (execute the cell w/ the function).

Given these statistical tests what would you recommend in terms of allocation of the ad budget? 

In [10]:
# use stats.norm.ppf([.025, .975], mean, std error)
# where mean is sample mean and std error is std dev / sqrt(obs) (from Central Limit Theorem)
fb['ctr'] = (fb['Clicks']/fb['Impressions'])

obs_916 = len(fb[fb.xyz_campaign_id == 916].ctr)
mean_916 = fb[fb.xyz_campaign_id == 916].ctr.mean()
stderror_916 = (fb[fb.xyz_campaign_id == 916].ctr.std())/(np.sqrt(obs_916))
stddev_916 = fb[fb.xyz_campaign_id == 916].ctr.std()

# confidence interval for ad 916
CI_916 = stats.norm.ppf([.025, .975], mean_916, stderror_916).round(6)

print([CI_916[0],CI_916[1]])


[0.000138, 0.000222]


In [11]:
# confidence interval for ad 936
obs_936 = len(fb[fb.xyz_campaign_id == 936].ctr)
mean_936 = fb[fb.xyz_campaign_id == 936].ctr.mean()
stderror_936 = (fb[fb.xyz_campaign_id == 936].ctr.std())/(np.sqrt(obs_936))
stddev_936 = fb[fb.xyz_campaign_id == 936].ctr.std()
CI_936 = stats.norm.ppf([.025, .975], mean_936, stderror_936).round(6)

print([CI_936[0],CI_936[1]])

[0.000151, 0.000179]


In [12]:
# confidence interval for ad 1178
obs_1178 = len(fb[fb.xyz_campaign_id == 1178].ctr)
mean_1178 = fb[fb.xyz_campaign_id == 1178].ctr.mean()
stderror_1178 = (fb[fb.xyz_campaign_id == 1178].ctr.std())/(np.sqrt(obs_1178))
stddev_1178 = fb[fb.xyz_campaign_id == 1178].ctr.std()
CI_1178 = stats.norm.ppf([.025, .975], mean_1178, stderror_1178).round(6)

print([CI_1178[0],CI_1178[1]])

[0.000157, 0.000167]


**Edit this cell**

1. **(write 1-2 sentences to compare the confidence interval of 916 vs 936's CTR)**The confidence intervals for 916 and 936 overlap, hence; there is no significant difference between them.
1. **(write 1-2 sentences to compare the confidence interval of 916 vs 1178's CTR)**The confidence intervals for 916 and 1178 overlap, hence; there is no significant difference between them

In [13]:
# Use this custom funciton

def ttest_2sample(m1,sd1,N1,m2,sd2,N2, twotail = True, equalvar = False):
    """
    This function requires you tu supply:
    m1: mean of sample 1
    sd1: std. dev of sample 1
    N1: number of obs of sample 1
    m2: mean of sample 2
    sd2: std dev of sample 2
    N2: number of obs of sample 2

    Optional inputs:
    twotail = True (default) / False. If False, then 1 tail
    equalvar = True / False (default). If True, assumes equal population variance.
    """

    # The difference between equal and unequal variance is only in how to compute
    # the test statistic and degree of freedom.
    if equalvar:
        spsquare = ((N1-1)*sd1**2+(N2-1)*sd2**2)/(N1+N2-2) 
        T = (m1-m2)/np.sqrt(spsquare*(1/N1+1/N2))
        nu = N1+N2-2
    else:
        nu = (sd1**2/N1+sd2**2/N2)**2/((sd1**2/N1)**2/(N1-1)+(sd2**2/N2)**2/(N2-1)) # new degree of freedom
        T = (m1-m2)/(np.sqrt(sd1**2/N1+sd2**2/N2))
    
    # If the first mean is bigger, we need to do 1- cdf
    # Otherwise just compute cdf
    if m1>m2:
        pval = 1-stats.t.cdf(T, df = nu)
    else:
        pval = stats.t.cdf(T, df = nu)

    # return p values
    # If 2 tail, we must multiply by 2
    # otherwise we just return the computed pval
    if twotail == True:
        return pval*2
    else:
        return pval

In [14]:
# run a 2 tail 2 sample t-test to compare the population CTRs for 936 and 916
ttest_2sample(mean_936,stddev_936,obs_936,mean_916,stddev_916,obs_916, twotail = True, equalvar = False)

0.5080964498604692

In [16]:
# run a 2 tail 2 sample t-test to compare the population CTRs for 936 and 1178
ttest_2sample(mean_936,stddev_936,obs_936,mean_1178,stddev_1178,obs_1178, twotail = True, equalvar = False)

0.7156337701785429

**Edit this cell to answer**

**(Write a few sentences to describe how to proceed with the allocation of the ad budget going forward. Be sure to use the 2 tail t-tests above to support your strategy)**

From the t-test performed, the p-values are greater than the significance level of 0.05. This would imply that there is no significant differences between the different campaigns.

# 3. (4 points)

Using the Starbucks dataset, conduct an appropriate statistical test to determine:
1. Is there any statistically appreciable difference in the redemption rates of BOGO vs. Discount promotions?
1. Among those who redeemed an offer and reported their income, is there any statistical difference in the average income associated with BOGO vs. discount redemptions? Note that not every observation reports income (select only the observations that do).

In [17]:
# read in the starbucks_promos.csv file as the dataframe variable named sb, use index_col = 0 option
sb = pd.read_csv('drive/MyDrive/MKTG3509/Data Sets/starbucks_promos.csv')

In [18]:
# preview first 5 rows to get a sense of the contents of the dataframe
# note that each observation represents an offer received by a customer
sb.head()

,Unnamed: 0,uid,event,time,gender,age,register_date,income,offer_id,offer_reward,channels,difficulty,duration,offer_type,offer_time,transaction_amount,redeem_time,redeemed
0,1,0020c2b971eb4e9188eac86d93036a77,offer received,0,F,59,20160304,90000.0,fafdcd668e3743c1bb461111dcafc2a4,2.0,"['web', 'email', 'mobile', 'social']",10.0,240.0,discount,0.0,17.63,54.0,1
1,4,005500a7188546ff8a767329a2f7c76a,offer received,0,M,56,20171209,47000.0,ae264e3637204a6fb9bb56bc8210ddfd,10.0,"['email', 'mobile', 'social']",10.0,168.0,bogo,0.0,NaN,NaN,0
2,5,0056df74b63b4298809f0b375a304cf4,offer received,0,M,54,20160821,91000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,"['web', 'email', 'mobile']",5.0,168.0,bogo,0.0,27.86,132.0,1
3,6,00715b6e55c3431cb56ff7307eb19675,offer received,0,F,58,20171207,119000.0,ae264e3637204a6fb9bb56bc8210ddfd,10.0,"['email', 'mobile', 'social']",10.0,168.0,bogo,0.0,27.26,12.0,1
4,8,00840a2ca5d2408e982d56544dc14ffd,offer received,0,M,26,20141221,61000.0,2906b810c7d4411798c6938adc9daaa5,2.0,"['web', 'email', 'mobile']",10.0,168.0,discount,0.0,6.05,540.0,1


In [19]:
# select the redemed column for discount offers, name this data the variable disc
disc = sb[sb.offer_type == "discount"].redeemed

#Redemption rate for discount offers
len(disc[disc == 1])/len(disc)

0.6066856562878564

In [20]:
# select the redemed column for buy one get one (bogo) offers, name this data the variable bogo
bogo = sb[sb.offer_type == "bogo"].redeemed

#Redemption rate for bogo offers
len(bogo[bogo == 1])/len(bogo)


0.5375586084789665

In [21]:
# use stats.ttest_ind to test whether the 2 types of offers yield different redemption rates
t,p =stats.ttest_ind(disc,bogo)
print("Test Statistic :", t.round(4))
print("P-Value :", p.round(4))


Test Statistic : 17.3016
P-Value : 0.0


In [22]:
# Select the income column for those who redeemed the BOGO offer, call this the variable bogo_income
# note you need to use 3 condtitions for the row selection: one for redeemed, one for bogo offer, and one for income is not missing
# FYI: dataframe.somecolumn.notnull() is the Boolean evaluation for whether the somecolumn is not missing data
bogo_income = sb[sb.offer_type == "bogo" ][sb.redeemed == 1][sb.income.notnull() == True].income


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [23]:
# Select the income column for those who redeemed the discount offer, call this the variable disc_income
# note you need to use 3 condtitions for the row selection: one for redeemed, one for discount offer,  and one for income is not missing
# FYI: dataframe.somecolumn.notnull() is the Boolean evaluation for whether the somecolumn is not missing data
disc_income = sb[sb.offer_type == "discount" ][sb.redeemed == 1][sb.income.notnull() == True].income


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [24]:
# What is the average income for bogo redeemers (bogo_income.mean())?
bogo_income.mean()


70114.8559593297

In [25]:
# What is the average income for discount redeemers?
disc_income.mean()


68617.39130434782

In [26]:
# use stats.ttest_ind to test whether the average income of the redeemers of the 2 offer types are different
t,p =stats.ttest_ind(disc_income,bogo_income)
print("Test Statistic :", t.round(4))
print("P-Value :", p.round(4))


Test Statistic : -6.3434
P-Value : 0.0


**Edit this cell**
1. **(Write a sentence to describe the relative redemption rates of the 2 offer types based on the result of the t-test).**

From the t-test performed, the p-value is very less than the significant level; therefore, the null hypothesis that there is no difference is rejected. The redemtion rates of the two offers differs.

The redemption rate for discount offers is relatively higher than those of bogo offers
1. **(Write a sentence to describe the relative incomes of the redeemers of the 2 offer types based on the result of the t-test).**

From the t-test performed, the p-value is very less than the significant level; therefore, the null hypothesis that there is no difference is rejected. The income of the two offer types are different. The income of bogo redeemers is relatively higher than the discount offer type redeemers.



**Citations:**
Received help from stackoverflow.com , realpython.com , geeksforgeeks.org , and pandas.pydata.org looking for similar examples of lines of code that related to concepts used for HW problems
